### Check the assigned GPU

In [1]:
!nvidia-smi

Thu Dec  4 20:47:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   69C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Install libraries

In [2]:
!pip install ConfigArgParse
!pip install torchtext==0.4.0
!apt-get install file autoconf libtool

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
autoconf is already the newest version (2.71-2).
libtool is already the newest version (2.4.6-15build2).
file is already the newest version (1:5.41-3ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


### Set environmental variables

In [3]:
import os
pwd = os.getcwd()
### This directory contains libraries and exexution files.
APPS_DIR = pwd + '/apps'
%env APPS_DIR=$APPS_DIR
### This directory contains training, development and test set
DATA_DIR = pwd + '/dataset'
%env DATA_DIR=$DATA_DIR
### This directory contains output directions.
WORK_DIR = pwd + '/work'
%env WORK_DIR=$WORK_DIR

#japanese -> english
%env LANG_PAIR=ja-en
#english -> japanese
#%env LANG_PAIR=en-ja

%env DATASET=$DATA_DIR/kftt-data-1.0/data/tok

env: APPS_DIR=/content/apps
env: DATA_DIR=/content/dataset
env: WORK_DIR=/content/work
env: LANG_PAIR=ja-en
env: DATASET=/content/dataset/kftt-data-1.0/data/tok


### Install applications

In [4]:
%%shell

echo "download and make apps..."

mkdir -p ${APPS_DIR}
cd ${APPS_DIR}

git clone https://github.com/moses-smt/mosesdecoder.git
git clone https://github.com/OpenNMT/OpenNMT-py.git -b 1.2.0
git clone https://github.com/neubig/kytea.git
cd kytea
autoreconf -i
./configure --prefix=${APPS_DIR}/kytea
make
make install

echo "finish apps preparation."

download and make apps...
fatal: destination path 'mosesdecoder' already exists and is not an empty directory.
fatal: destination path 'OpenNMT-py' already exists and is not an empty directory.
fatal: destination path 'kytea' already exists and is not an empty directory.
libtoolize: Consider adding '-I m4' to ACLOCAL_AMFLAGS in Makefile.am.
configure.ac:28: warning: The macro `AC_PROG_LIBTOOL' is obsolete.
configure.ac:28: You should run autoupdate.
m4/libtool.m4:99: AC_PROG_LIBTOOL is expanded from...
configure.ac:28: the top level
checking for a BSD-compatible install... /usr/bin/install -c
checking whether build environment is sane... yes
checking for a race-free mkdir -p... /usr/bin/mkdir -p
checking for gawk... no
checking for mawk... mawk
checking whether make sets $(MAKE)... yes
checking whether make supports nested variables... yes
checking whether make supports the include directive... yes (GNU style)
checking for gcc... gcc
checking whether the C compiler works... yes
checkin

### Download data

In [5]:
%%shell

mkdir -p $WORK_DIR/input
mkdir -p ${DATA_DIR}

# Download data set
wget -O ${DATA_DIR}/kftt-data-1.0.tar.gz http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz
cd ${DATA_DIR}
# Uncompress
tar xvzf kftt-data-1.0.tar.gz

paste \
  ${DATASET}/kyoto-train.cln.en \
  ${DATASET}/kyoto-train.cln.ja |
  shuf > ${DATASET}/train.shuf

#1p
#head -n $((`cat ${DATASET}/train.shuf | wc -l`/100)) ${DATASET}/train.shuf | cut -f 1 > ${DATASET}/train.en
#head -n $((`cat ${DATASET}/train.shuf | wc -l`/100)) ${DATASET}/train.shuf | cut -f 2 > ${DATASET}/train.ja
#20p
head -n $((`cat ${DATASET}/train.shuf | wc -l`/5)) ${DATASET}/train.shuf | cut -f 1 > ${DATASET}/train.en
head -n $((`cat ${DATASET}/train.shuf | wc -l`/5)) ${DATASET}/train.shuf | cut -f 2 > ${DATASET}/train.ja
#60p
#head -n $((3*`cat ${DATASET}/train.shuf | wc -l`/5)) ${DATASET}/train.shuf | cut -f 1 > ${DATASET}/train.en
#head -n $((3*`cat ${DATASET}/train.shuf | wc -l`/5)) ${DATASET}/train.shuf | cut -f 2 > ${DATASET}/train.ja
#100p
#cut -f 1 ${DATASET}/train.shuf > ${DATASET}/train.en
#cut -f 2 ${DATASET}/train.shuf > ${DATASET}/train.ja

--2025-12-04 20:47:40--  http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz
Resolving www.phontron.com (www.phontron.com)... 173.236.247.185
Connecting to www.phontron.com (www.phontron.com)|173.236.247.185|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99246893 (95M) [application/gzip]
Saving to: ‘/content/dataset/kftt-data-1.0.tar.gz’

/content/dataset/kf 100%[===================>]  94.65M  17.9MB/s    in 6.9s    

2025-12-04 20:47:48 (13.7 MB/s) - ‘/content/dataset/kftt-data-1.0.tar.gz’ saved [99246893/99246893]

kftt-data-1.0/
kftt-data-1.0/data/
kftt-data-1.0/data/orig/
kftt-data-1.0/data/orig/kyoto-tune.en
kftt-data-1.0/data/orig/kyoto-dev.ja
kftt-data-1.0/data/orig/kyoto-dev.en
kftt-data-1.0/data/orig/kyoto-train.en
kftt-data-1.0/data/orig/kyoto-tune.ja
kftt-data-1.0/data/orig/kyoto-train.ja
kftt-data-1.0/data/orig/kyoto-test.ja
kftt-data-1.0/data/orig/kyoto-test.en
kftt-data-1.0/data/tok/
kftt-data-1.0/data/tok/kyoto-tune.en
kftt-data-1.0/data/

### See sample sentences in the data

In [6]:
%%shell

train_name=train
src=`echo ${LANG_PAIR} | awk -F"-" '{print $1}'`
trg=`echo ${LANG_PAIR} | awk -F"-" '{print $2}'`

sed -n 1P ${DATASET}/${train_name}.${src}
sed -n 1P ${DATASET}/${train_name}.${trg}
sed -n 2P ${DATASET}/${train_name}.${src}
sed -n 2P ${DATASET}/${train_name}.${trg}

1923 年 （ 大正 12 年 ） 6 月 1 日 、 先駆 的 映画 監督 で あ る 牧野 省三 が 設立 し た 。
The company was founded by Shozo MAKINO , the pioneering director of Japanese films , on June 1 , 1923 .
さらに 各 焼成 室 に は 薪 を 投入 する ため の 「 小口 」 と 呼 ば れ る 小さな 穴 が 設け られ て い る の が 普通 で あ る 。
A small hole called a " Koguchi " is made in each baking chamber for adding more firewood .


### Preprocessing of OpenNMT-py

In [7]:
%%shell

cd ${APPS_DIR}/OpenNMT-py
suffix="en-ja ja-en"
train_name=train
src=`echo ${LANG_PAIR} | awk -F"-" '{print $1}'`
trg=`echo ${LANG_PAIR} | awk -F"-" '{print $2}'`
    python preprocess.py \
    -train_src ${DATASET}/${train_name}.${src} \
    -train_tgt ${DATASET}/${train_name}.${trg} \
    -valid_src ${DATASET}/kyoto-dev.${src} \
    -valid_tgt ${DATASET}/kyoto-dev.${trg} \
    -save_data ${DATA_DIR}/dicts-${train_name}-${LANG_PAIR} \
    -src_words_min_frequency 5 \
    -tgt_words_min_frequency 5 \
    -src_seq_length 40 \
    -tgt_seq_length 40
#train_src, train_tgt, valid_src, valid_tgt: text
#save_data: bin

/content/apps/OpenNMT-py/onmt/modules/sparse_activations.py:47: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/apps/OpenNMT-py/onmt/modules/sparse_activations.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/content/apps/OpenNMT-py/onmt/modules/sparse_losses.py:12: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/apps/OpenNMT-py/onmt/modules/sparse_losses.py:36: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/content/apps/OpenNMT-py/onmt/models/sru.py:396: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.cust

### Check the preprocessed data

In [8]:
%%shell
cd ${APPS_DIR}
rm -rf OpenNMT-py
git clone https://github.com/OpenNMT/OpenNMT-py.git -b 1.2.0

# 正确添加 weights_only=False
find OpenNMT-py -name "*.py" -exec sed -i 's/torch\.load(\([^)]*\))/torch.load(\1, weights_only=False)/g' {} \;

Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 21869, done.
remote: Counting objects: 100% (1017/1017), done.
remote: Compressing objects: 100% (395/395), done.
remote: Total 21869 (delta 873), reused 622 (delta 622), pack-reused 20852 (from 3)
Receiving objects: 100% (21869/21869), 314.06 MiB | 16.31 MiB/s, done.
Resolving deltas: 100% (15626/15626), done.
Note: switching to '60125c807d1cb18099a69dbfba699bcdf30560b1'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



In [9]:
%cd {APPS_DIR}/OpenNMT-py
import torch

# Training data
data = torch.load('/content/dataset/dicts-train-ja-en.train.0.pt', weights_only=False)

# Vocabulary
vocab = torch.load('/content/dataset/dicts-train-ja-en.vocab.pt', weights_only=False)

print("Sentence number of training data: ", len(data))
print("")
print("data[0]")
print("index: ", data[0].indices)
print("Source: ", data[0].src)
print("Translated: ", data[0].tgt)
print("")
print("data[1]")
print("index: ", data[1].indices)
print("Source: ", data[1].src)
print("Translated: ", data[1].tgt)
print("")
print("The above input is transformed into vectors of indices as follows:")
print(vocab['src'].fields[0][1].process((data[0].src[0],data[1].src[0])))

/content/apps/OpenNMT-py


/content/apps/OpenNMT-py/onmt/modules/sparse_activations.py:47: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/apps/OpenNMT-py/onmt/modules/sparse_activations.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/content/apps/OpenNMT-py/onmt/modules/sparse_losses.py:12: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/apps/OpenNMT-py/onmt/modules/sparse_losses.py:36: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


Sentence number of training data:  65976

data[0]
index:  0
Source:  [['くつき', '温泉', 'てん', 'く', 'う']]
Translated:  [['Kutsuki', 'Onsen', 'Tencoo', '(', 'KUTSUKI', 'KUR-RESORT', 'TENCOO', ')']]

data[1]
index:  1
Source:  [['左京', '大夫', '信実', '（', '藤原', '信実', '）']]
Translated:  [['Sakyo', 'no', 'Daibu', '(', 'Master', 'of', 'the', 'Eastern', 'Capital', 'Offices', ')', 'Nobuzane', '(', 'FUJIWARA', 'no', 'Nobuzane', ')']]

The above input is transformed into vectors of indices as follows:
(tensor([[    0,  1257],
        [  736,   704],
        [ 3266, 11454],
        [   35,    16],
        [   33,   105],
        [    1, 11454],
        [    1,    17]]), tensor([5, 7]))


### Run training

In [10]:
%%shell

start_time=`date +%s`

TIME_PATH=$(pwd)/train.time.log



TRAIN_NAME=train
MODEL_DIR=${WORK_DIR}/models

NUM_EPOCH=10

mkdir -p ${MODEL_DIR}

NUM_DATA=$(cat ${DATASET}/${TRAIN_NAME}.en | wc -l)

cd ${APPS_DIR}/OpenNMT-py

batch_size=256
num_steps=$((${NUM_DATA}*${NUM_EPOCH}/${batch_size}))
# seq2seq with attention (LSTM-based)
python train.py \
    -data ${DATA_DIR}/dicts-${TRAIN_NAME}-${LANG_PAIR} \
    -save_model ${MODEL_DIR}/${LANG_PAIR} \
    -layers 2 \
    -rnn_size 500 \
    -word_vec_size 300 \
    -optim adam \
    -learning_rate 0.001 \
    -dropout 0.3 \
    -batch_size ${batch_size} \
    -report_every 1 \
    -save_checkpoint_steps ${num_steps} \
    -train_steps ${num_steps} \
    -gpu_rank 0 |& awk '(NR%30==0 || NR < 70){print}' # Show partial logs
cp ${MODEL_DIR}/${LANG_PAIR}_step_${num_steps}.pt ${MODEL_DIR}/${LANG_PAIR}_final.pt


end_time=`date +%s`
time=$((end_time - start_time))
echo "${time} (sec)" >& $TIME_PATH

/content/apps/OpenNMT-py/onmt/modules/sparse_activations.py:47: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/apps/OpenNMT-py/onmt/modules/sparse_activations.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/content/apps/OpenNMT-py/onmt/modules/sparse_losses.py:12: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/apps/OpenNMT-py/onmt/modules/sparse_losses.py:36: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/content/apps/OpenNMT-py/onmt/models/sru.py:396: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.cust

### Prediction (Translation)

In [14]:
!sed -i 's/torch\.load(model_path,/torch.load(model_path, weights_only=False,/' /content/apps/OpenNMT-py/onmt/model_builder.py

In [15]:
%%shell


MODELS=${WORK_DIR}/models
OUT_DIR=${WORK_DIR}/outputs
USRDIR=${WORK_DIR}/input
SCORE_PATH=$(pwd)/score.txt

mkdir -p ${OUT_DIR}

cd ${APPS_DIR}/OpenNMT-py

src=`echo ${LANG_PAIR} | awk -F"-" '{print $1}'`
trg=`echo ${LANG_PAIR} | awk -F"-" '{print $2}'`

# Translating sentences by greedy decoding
python translate.py \
    -model ${MODELS}/${LANG_PAIR}_final.pt \
    -src ${DATASET}/kyoto-test.${src} \
    -output ${OUT_DIR}/test.${trg} \
    -gpu 0 \
    -beam_size 1 \
    -batch_size 512 \
    -verbose

流式输出内容被截断，只能显示最后 5000 行内容。
SENT 161: ['国鉄', 'D50', '形', '蒸気', '機関', '車']
PRED 161: JNR / steam locomotives steam locomotive
PRED SCORE: -0.0198

[2025-12-04 21:00:39,570 INFO] 
SENT 162: ['1925', '年', '日立', '製作', '所', '製', '。']
PRED 162: 1925 : <unk> <unk> Co . , Ltd.
PRED SCORE: -1.4770

[2025-12-04 21:00:39,570 INFO] 
SENT 163: ['直方', '機関', '区', 'より', '転入', '。']
PRED 163: It is a <unk> ( a facility of building and a facility ) .
PRED SCORE: -0.0002

[2025-12-04 21:00:39,570 INFO] 
SENT 164: ['1979', '年', '車', '籍', '抹消', '。']
PRED 164: The <unk> <unk> was <unk> .
PRED SCORE: -0.0003

[2025-12-04 21:00:39,570 INFO] 
SENT 165: ['国鉄', 'D51', '形', '蒸気', '機関', '車']
PRED 165: JNR / steam locomotives steam locomotive
PRED SCORE: -0.0198

[2025-12-04 21:00:39,570 INFO] 
SENT 166: ['1936', '年', '川崎', '車輌', '製', '。']
PRED 166: 1936 : <unk> <unk> .
PRED SCORE: -0.0001

[2025-12-04 21:00:39,570 INFO] 
SENT 167: ['浜田', '機関', '区', 'より', '転入', '。']
PRED 167: It is located in the second section of Ha

### Evaluation (Copy the output and report it)

In [16]:
%%shell


MODELS=${WORK_DIR}/models
OUT_DIR=${WORK_DIR}/outputs
USRDIR=${WORK_DIR}/input
SCORE_PATH=$(pwd)/score.txt

cd ${APPS_DIR}/OpenNMT-py

src=`echo ${LANG_PAIR} | awk -F"-" '{print $1}'`
trg=`echo ${LANG_PAIR} | awk -F"-" '{print $2}'`

for i in 1 2 3 4 5; do
  echo [Generated translation $i]
  sed -n ${i}P ${OUT_DIR}/test.${trg}
  echo [Reference translation $i]
  sed -n ${i}P ${DATASET}/kyoto-test.${trg}
done

# Calculate BLEU score
perl ${APPS_DIR}/mosesdecoder/scripts/generic/multi-bleu.perl \
    ${DATASET}/kyoto-test.${trg} \
    < ${OUT_DIR}/test.${trg} \
    1> ${OUT_DIR}/result_${LANG_PAIR}.bleu \
    2> /dev/null

cat ${OUT_DIR}/result_${LANG_PAIR}.bleu | sed -r 's/(BLEU = [0-9]*\.[0-9]*), .*/\1/g' | tee ${SCORE_PATH} > /dev/null
echo "------------------------------"
nvidia-smi
cat train.time.log
cat score.txt

[Generated translation 1]
<unk> <unk>
[Reference translation 1]
Infobox Buddhist
[Generated translation 2]
Dogen ( <unk> ) was a Zen priest in the early Kamakura period .
[Reference translation 2]
Dogen was a Zen monk in the early Kamakura period .
[Generated translation 3]
Founder of the Soto sect .
[Reference translation 3]
The founder of Soto Zen
[Generated translation 4]
He was also called <unk> <unk> in his later years .
[Reference translation 4]
Later in his life he also went by the name Kigen .
[Generated translation 5]
In the <unk> , it is called <unk> .
[Reference translation 5]
Within the sect he is referred to by the honorary title Koso .
------------------------------
Thu Dec  4 21:01:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| G

### Advanced: translating your text (no obligation to run)
#### You can edit the below part `echo 何らかの文章をここに書く。 > ${USRDIR}/user.${src}` to translate your own Japanese text.

In [17]:
%%shell
MODELS=${WORK_DIR}/models
OUT_DIR=${WORK_DIR}/outputs
USRDIR=${WORK_DIR}/input
SCORE_PATH=$(pwd)/score.txt
src=`echo ${LANG_PAIR} | awk -F"-" '{print $1}'`
trg=`echo ${LANG_PAIR} | awk -F"-" '{print $2}'`

echo 何らかの文章をここに書く。 > ${USRDIR}/user.${src}

if [ ! -e ${USRDIR}/user.${src} ]
then
    echo "${USRDIR}/user.${src} does not exist."
    exit
fi

# Tokenization
if [ ${src} = "ja" -a ${src} != "en" ]
then
    # Tokenize Japanese sentences
    bash ${APPS_DIR}/kytea/src/bin/kytea \
    < ${USRDIR}/user.${src} |\
    sed 's/\/[^ ]\+//g' \
    > ${USRDIR}/user.tok.${src}
elif [ ${src} = "en" -a ${src} != "ja" ]
then
    # Tokenize English sentences
    perl ${APPS_DIR}/mosesdecoder/scripts/tokenizer/tokenizer.perl \
    -l en \
    < ${USRDIR}/user.${src} \
    > ${USRDIR}/user.tok.${src}
else
echo "Language: ${src} is undefined."
    continue
fi

# Tokenized text
cat ${USRDIR}/user.tok.${src}

# Do translation
python translate.py \
    -model ${MODELS}/${LANG_PAIR}_final.pt \
    -src ${USRDIR}/user.tok.${src} \
    -output ${OUT_DIR}/user.${trg} \
    -gpu 0 \
    -beam_size 1 \
    -batch_size 512 \
    -verbose

何 ら か の 文章 を ここ に 書 く 。
/content/apps/OpenNMT-py/onmt/modules/sparse_activations.py:47: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/apps/OpenNMT-py/onmt/modules/sparse_activations.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/content/apps/OpenNMT-py/onmt/modules/sparse_losses.py:12: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/apps/OpenNMT-py/onmt/modules/sparse_losses.py:36: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/content/apps/OpenNMT-py/onmt/models/sru.py:396: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Pl